In [1]:
import pickle
import torch
import io
import os

In [2]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [3]:
directory = os.fsencode(".")
data_dict = dict()
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".pkl"):
        with open(os.path.join(os.fsdecode(directory), filename), 'rb') as f:
            while True:
                try:
                    data = CPU_Unpickler(f).load()
                except EOFError:
                    break
                data_dict[("GIN", data[1][0], data[-1].loss, data[-1].loss_parameter)] = data[:-1]
print(data_dict.keys())

dict_keys([('GIN', 'CCoHG', 'CosineEmbeddingLoss', 1)])


C:\Users\billy\AppData\Local\Temp\ipykernel_22036\3158114827.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return lambda b: torch.load(io.BytesIO(b), map_location='cpu

In [5]:
keys = data_dict.keys()
counts = dict()
for key in keys:
    value = data_dict[key]
    counts[key] = sum([1 for x in value if x[2] and x[4]])
    print(f"{counts[key]}/{len(value)}", key)

0/100 ('GIN', 'CCoHG', 'CosineEmbeddingLoss', 1)
